In [1]:
%load_ext autoreload
%autoreload
import os, json
from types import SimpleNamespace
from experiment import run_model
from eval import calculate_stats
import pickle
from datetime import datetime 
import torch
from transformers import BertTokenizer
from sklearn.metrics import classification_report
from util.tools import load_config, configEncoder

In [2]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [3]:
config_folder = "config/"
config = load_config(config_folder)
config.modelconfig.DROP_OUT

0.01

In [4]:
train_inputs, train_label_ids, train_preds, train_loss, dev_inputs, dev_label_ids, dev_loss, dev_preds,test_inputs, test_preds, \
test_labels, test_loss  = run_model(config, device)

INFO:root:  Num examples = 1000
INFO:root:  Batch size = 12
INFO:root:  Num steps = 249


Looking for cached feature pickle file data_divided/train_features_256.pkl


Epoch:   0%|                                                                                     | 0/3 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "



 predicted: tensor([5, 4, 7, 5, 5, 5, 0, 7, 3, 0, 5, 7]) 
 true:  tensor([8, 8, 6, 0, 6, 0, 3, 8, 8, 3, 5, 8])
 loss 2.220604
 Accuumulated for ephoch, loss:  26.647250175476074  , corrects: tensor(1)  size:  1000

 predicted: tensor([5, 3, 5, 3, 5, 5, 0, 5, 5, 7, 5, 7]) 
 true:  tensor([8, 8, 8, 0, 2, 8, 3, 5, 8, 7, 5, 5])
 loss 2.197948
 Accuumulated for ephoch, loss:  79.3979959487915  , corrects: tensor(4)  size:  1000

 predicted: tensor([5, 6, 5, 1, 6, 5, 5, 7, 0, 5, 5, 7]) 
 true:  tensor([5, 6, 5, 8, 8, 0, 7, 3, 3, 3, 1, 8])
 loss 2.190070
 Accuumulated for ephoch, loss:  158.2405128479004  , corrects: tensor(7)  size:  1000

 predicted: tensor([5, 5, 7, 5, 7, 4, 3, 0, 6, 4, 5, 7]) 
 true:  tensor([6, 8, 8, 8, 3, 8, 8, 3, 5, 8, 5, 8])
 loss 2.224390
 Accuumulated for ephoch, loss:  265.01122283935547  , corrects: tensor(8)  size:  1000

 predicted: tensor([7, 5, 5, 5, 5, 0, 5, 7, 7, 0, 3, 1]) 
 true:  tensor([8, 8, 0, 1, 0, 1, 8, 2, 1, 6, 6, 8])
 loss 2.219349
 Accuumulated fo

...

Training complete
Best val Acc: 0.689000


INFO:root:  Num examples = 500
INFO:root:  Batch size = 12
INFO:root:  Num steps = 123



Looking for cached feature pickle file data_divided/dev_features_256.pkl


INFO:root:  Num examples = 1000
INFO:root:  Batch size = 12
INFO:root:  Num steps = 249



Looking for cached feature pickle file data_divided/test_features_256.pkl


In [7]:
# train_inputs, train_label_ids, train_preds, train_loss, dev_inputs, dev_label_ids, dev_loss, dev_preds, test_preds, test_labels

In [5]:
print(classification_report(test_labels,test_preds ))

              precision    recall  f1-score   support

           0       0.28      0.86      0.42        44
           1       0.00      0.00      0.00        53
           2       0.44      0.96      0.61        50
           3       0.32      0.31      0.31        55
           4       0.00      0.00      0.00        10
           5       0.35      0.30      0.32        57
           6       0.00      0.00      0.00        46
           7       0.00      0.00      0.00        33
           8       1.00      1.00      1.00       652

    accuracy                           0.77      1000
   macro avg       0.27      0.38      0.30      1000
weighted avg       0.72      0.77      0.74      1000



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [2]:
def save_missed_cases_to_file(file_start_name, dev_preds, dev_label_ids, train_inputs):
    tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
    missed_cases = []
    for i in range(0,50):
        if dev_label_ids[i] !=  dev_preds[i]:
             missed_cases.append([ dev_preds[i],  dev_label_ids[i] , " ". join (tokenizer.convert_ids_to_tokens(train_inputs[i])) ])

    #Save into a file
    missed_cases_file = config.programsettings.REPORTS_DIR +file_start_name + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
    with open(missed_cases_file, "wb") as f:
        pickle.dump(missed_cases, f)  
        
save_missed_cases_to_file("BIOBERT_fc_missedcases_" , dev_preds, dev_label_ids, train_inputs)

NameError: name 'dev_preds' is not defined

In [8]:
import numpy as np
np.array(dev_label_ids).shape, np.array(dev_preds).shape, np.array(train_preds).shape, np.array(train_label_ids).shape

((99,), (99,), (99,), (99,))

In [9]:
%autoreload
train_mcc, train_f1_score, train_df_results, train_label_matches_df = calculate_stats(train_label_ids,train_preds )
dev_mcc, dev_f1_score, dev_df_results, dev_label_matches_df = calculate_stats(dev_label_ids,dev_preds )


 label: [0 3 2 5 4 6 2 5 0 1 5 5 1 0 5 3 4 6 6 2 7 6 5 0 0 6 0 1 0 6 0 6 0 3 1 1 4
 2 4 3 0 1 0 2 2 2 3 5 7 6 3 6 1 4 3 0 3 2 0 0 1 3 6 4 6 6 2 1 5 5 2 6 6 3
 0 0 5 4 2 0 1 2 1 0 4 0 0 2 2 1 0 2 1 1 0 6 5 4 6]

 preds: [6 6 4 6 4 4 4 7 7 4 3 4 6 4 6 4 4 4 6 4 7 6 6 4 4 4 4 4 7 6 7 6 4 6 4 6 4
 6 4 4 6 6 6 6 6 4 6 6 7 6 4 6 4 4 4 6 6 6 6 6 6 4 6 4 6 6 6 6 6 4 4 6 6 4
 6 6 4 6 6 6 6 4 3 6 4 3 6 6 6 4 6 4 4 4 4 4 4 4 7]

 label: [6 2 8 2 8 8 8 0 0 3 5 0 8 1 8 3 8 8 8 8 0 0 8 8 8 8 6 8 2 7 3 8 3 6 0 2 0
 6 5 0 8 1 6 1 3 0 7 0 0 8 8 8 8 8 0 8 4 2 2 8 0 5 1 2 8 8 8 0 5 8 4 2 8 0
 8 0 8 8 8 3 8 8 4 5 2 5 2 0 3 8 0 8 8 8 8 8 8 8 1]

 preds: [6 6 7 4 7 7 4 4 6 4 6 6 4 6 7 4 7 6 6 4 6 4 7 7 6 7 6 6 6 7 4 4 6 6 3 4 4
 7 4 4 4 6 4 6 6 4 4 6 4 6 7 4 4 7 6 7 4 4 4 7 4 6 4 4 7 6 7 4 6 7 4 6 6 4
 7 6 4 7 7 6 7 6 6 6 7 6 4 4 4 6 4 7 7 6 3 4 7 4 4]


In [10]:
all_experiment_results = []
all_experiment_results.append([config, train_loss, dev_loss, train_mcc, train_f1_score,dev_mcc,dev_f1_score, 
                               dev_label_ids, dev_preds,train_label_ids,train_preds  ])
all_experiment_results

[[<util.tools.config at 0x2ea8c1b7a30>,
  2.1800858974456787,
  2.2085455417633058,
  0.1305070354677441,
  0.13842203548085902,
  0.03532304615650811,
  0.03616457816999822,
  array([6, 2, 8, 2, 8, 8, 8, 0, 0, 3, 5, 0, 8, 1, 8, 3, 8, 8, 8, 8, 0, 0,
         8, 8, 8, 8, 6, 8, 2, 7, 3, 8, 3, 6, 0, 2, 0, 6, 5, 0, 8, 1, 6, 1,
         3, 0, 7, 0, 0, 8, 8, 8, 8, 8, 0, 8, 4, 2, 2, 8, 0, 5, 1, 2, 8, 8,
         8, 0, 5, 8, 4, 2, 8, 0, 8, 0, 8, 8, 8, 3, 8, 8, 4, 5, 2, 5, 2, 0,
         3, 8, 0, 8, 8, 8, 8, 8, 8, 8, 1], dtype=int64),
  array([6, 6, 7, 4, 7, 7, 4, 4, 6, 4, 6, 6, 4, 6, 7, 4, 7, 6, 6, 4, 6, 4,
         7, 7, 6, 7, 6, 6, 6, 7, 4, 4, 6, 6, 3, 4, 4, 7, 4, 4, 4, 6, 4, 6,
         6, 4, 4, 6, 4, 6, 7, 4, 4, 7, 6, 7, 4, 4, 4, 7, 4, 6, 4, 4, 7, 6,
         7, 4, 6, 7, 4, 6, 6, 4, 7, 6, 4, 7, 7, 6, 7, 6, 6, 6, 7, 6, 4, 4,
         4, 6, 4, 7, 7, 6, 3, 4, 7, 4, 4], dtype=int64),
  array([0, 3, 2, 5, 4, 6, 2, 5, 0, 1, 5, 5, 1, 0, 5, 3, 4, 6, 6, 2, 7, 6,
         5, 0, 0, 6, 0, 1, 0, 6, 0, 

In [11]:
dev_label_matches_df

predicted
labels         matched           
ADE-Drug       False            1
               True             1
Dosage-Drug    False            2
               True             3
Duration-Drug  False            1
               True             2
Form-Drug      False            6
Frequency-Drug False            7
Reason-Drug    False           18
Route-Drug     False            5
Strength-Drug  False           10
no relation    False           43

##  Try with BERT Sequential configuration

### Just change model from BIOR to BERT Sequence

In [12]:
config.programsettings.MODEL_NAME = "BERT_Sequence"
config.programsettings.DEBUG_PRINT = 0

In [13]:
train_inputs, train_label_ids, train_preds, train_loss, dev_inputs, dev_label_ids, dev_loss, dev_preds = run_model(config, device)

INFO:root:***** Running training *****
INFO:root:  Num examples = 99
INFO:root:  Batch size = 24
INFO:root:  Num steps = 40
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.modeling:extracting archive file cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c to temp dir C:\Users\pnars\AppData\Local\Temp\tmp_g5gm1j3
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size":

C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epoch:  10%|██████████▊                                                                                                 | 1/10 [01:26<12:56, 86.26s/it]


so far epoch accuracy:  0.1111111111111111


Epoch:  20%|█████████████████████▌                                                                                      | 2/10 [02:53<11:31, 86.47s/it]

Epoch:  30%|████████████████████████████████▍                                                                           | 3/10 [04:24<10:14, 87.77s/it]

...


so far epoch accuracy:  0.2222222222222222


Epoch:  90%|█████████████████████████████████████████████████████████████████████████████████████████████████▏          | 9/10 [13:08<01:27, 87.32s/it]

Epoch: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [14:37<00:00, 87.71s/it]


Training complete
Best val Acc: 0.222222


INFO:root:***** Running training *****
INFO:root:  Num examples = 99
INFO:root:  Batch size = 24
INFO:root:  Num steps = 40


In [14]:

train_mcc, train_f1_score, train_df_results, train_label_matches_df = calculate_stats(train_label_ids,train_preds )
dev_mcc, dev_f1_score, dev_df_results, dev_label_matches_df = calculate_stats(dev_label_ids,dev_preds)


 label: [1 3 4 6 2 4 5 2 6 2 6 4 1 4 3 1 4 0 0 5 5 0 1 0 0 1 0 6 6 3 3 5 6 0 1 2 6
 6 1 5 2 5 6 7 5 3 0 4 3 2 6 0 0 6 0 1 0 2 1 0 0 1 3 1 1 6 4 0 4 0 0 3 2 6
 1 5 6 3 2 2 2 0 3 5 2 0 2 1 0 7 0 0 5 2 2 6 6 4 5]

 preds: [4 4 6 6 4 4 5 6 6 6 6 4 5 4 4 6 6 4 6 5 5 6 4 6 6 6 6 6 4 4 4 5 6 5 6 5 4
 6 4 6 6 6 5 6 5 4 6 4 6 6 6 6 6 6 4 5 6 4 6 6 6 4 6 5 4 4 4 4 6 6 6 6 6 6
 6 4 6 6 4 5 4 6 4 4 4 6 5 5 6 6 6 4 6 4 5 4 5 4 5]

 label: [8 8 0 8 8 8 8 8 0 0 8 8 0 8 0 4 6 2 8 8 0 8 8 3 2 8 8 2 3 8 5 2 5 8 0 6 1
 5 3 0 8 4 8 2 7 8 3 8 8 0 8 8 8 0 8 0 6 8 5 8 0 8 0 3 8 0 0 8 8 1 8 8 8 3
 5 8 1 8 2 0 8 6 4 6 8 2 2 8 0 5 2 1 2 1 7 0 8 8 3]

 preds: [4 3 2 4 6 4 3 4 4 6 6 3 2 4 6 4 6 2 4 6 4 3 6 6 4 6 4 6 2 3 4 4 4 3 6 6 6
 6 6 6 6 4 4 6 2 4 6 4 6 6 4 4 4 6 4 5 6 3 6 6 6 3 4 2 4 6 2 4 4 6 6 3 4 2
 6 3 6 4 4 4 3 6 6 6 4 6 6 3 6 6 6 6 6 6 6 6 6 4 6]


In [15]:
all_experiment_results.append([config, train_loss, dev_loss, train_mcc, train_f1_score,dev_mcc,dev_f1_score, 
                               dev_label_ids, dev_preds,train_label_ids,train_preds  ])
all_experiment_results

[[<util.tools.config at 0x2ea8c1b7a30>,
  2.1800858974456787,
  2.2085455417633058,
  0.1305070354677441,
  0.13842203548085902,
  0.03532304615650811,
  0.03616457816999822,
  array([6, 2, 8, 2, 8, 8, 8, 0, 0, 3, 5, 0, 8, 1, 8, 3, 8, 8, 8, 8, 0, 0,
         8, 8, 8, 8, 6, 8, 2, 7, 3, 8, 3, 6, 0, 2, 0, 6, 5, 0, 8, 1, 6, 1,
         3, 0, 7, 0, 0, 8, 8, 8, 8, 8, 0, 8, 4, 2, 2, 8, 0, 5, 1, 2, 8, 8,
         8, 0, 5, 8, 4, 2, 8, 0, 8, 0, 8, 8, 8, 3, 8, 8, 4, 5, 2, 5, 2, 0,
         3, 8, 0, 8, 8, 8, 8, 8, 8, 8, 1], dtype=int64),
  array([6, 6, 7, 4, 7, 7, 4, 4, 6, 4, 6, 6, 4, 6, 7, 4, 7, 6, 6, 4, 6, 4,
         7, 7, 6, 7, 6, 6, 6, 7, 4, 4, 6, 6, 3, 4, 4, 7, 4, 4, 4, 6, 4, 6,
         6, 4, 4, 6, 4, 6, 7, 4, 4, 7, 6, 7, 4, 4, 4, 7, 4, 6, 4, 4, 7, 6,
         7, 4, 6, 7, 4, 6, 6, 4, 7, 6, 4, 7, 7, 6, 7, 6, 6, 6, 7, 6, 4, 4,
         4, 6, 4, 7, 7, 6, 3, 4, 7, 4, 4], dtype=int64),
  array([0, 3, 2, 5, 4, 6, 2, 5, 0, 1, 5, 5, 1, 0, 5, 3, 4, 6, 6, 2, 7, 6,
         5, 0, 0, 6, 0, 1, 0, 6, 0, 

In [16]:
dev_label_matches_df

predicted
labels         matched           
ADE-Drug       False            2
Dosage-Drug    True             5
Duration-Drug  False            1
               True             2
Form-Drug      False            6
Frequency-Drug False            7
Reason-Drug    False           18
Route-Drug     False            5
Strength-Drug  False            9
               True             1
no relation    False           43

In [17]:

all_model_results_pickle_file = config.programsettings.REPORTS_DIR + "multi_model_experiment_results_" + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
with open(all_model_results_pickle_file, "wb") as f:
    pickle.dump(all_experiment_results, f)  


In [18]:
print(classification_report(dev_label_ids,dev_preds ))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        18
           1       0.00      0.00      0.00         5
           2       0.12      0.10      0.11        10
           3       0.00      0.00      0.00         7
           4       0.06      0.67      0.11         3
           5       0.00      0.00      0.00         6
           6       0.11      1.00      0.20         5
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00        43

    accuracy                           0.08        99
   macro avg       0.03      0.20      0.05        99
weighted avg       0.02      0.08      0.02        99



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
# Save missed cases.
save_missed_cases_to_file("BERT_sequential_missedcases_" , dev_preds, dev_label_ids, train_inputs)


# Hyper paramter tuning based on experiments in experiments_batch

In [20]:
# import torch
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# # device = "cpu"
# from  experiments_batch import run_all_experiments_save
# run_all_experiments_save(device)